In [9]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models

# Load and preprocess the dataset
#def load_images(img_dir, img_height, img_width):
#    images = []

#    for filename in os.listdir(img_dir):
#        img_path = os.path.join(img_dir, filename)

#        img = cv2.imread(img_path)
#        img = cv2.resize(img, (img_width, img_height))

#        images.append(img / 255.0)

#    return np.array(images)

def load_images_from_video(video_path, img_height, img_width):
    cap = cv2.VideoCapture(video_path)
    images = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.resize(frame, (img_width, img_height))
        images.append(frame / 255.0)

    cap.release()
    return np.array(images)


# Define the autoencoder model
def build_autoencoder(input_shape):
    input_img = layers.Input(shape=input_shape)

    # Encoder
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    encoded = layers.MaxPooling2D((2, 2), padding='same')(x)

    # Decoder
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(encoded)
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)
    decoded = layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

    autoencoder = models.Model(input_img, decoded)
    autoencoder.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return autoencoder

# Calculate the steering angle
def calculate_steering_angle(lane_image):
    # Apply Gaussian Blur
    blurred = cv2.GaussianBlur(lane_image, (5, 5), 0)

    # Apply Canny Edge Detector
    edges = cv2.Canny(blurred, 50, 150)

    # Find the coordinates of the lane lines
    histogram = np.sum(edges[edges.shape[0]//2:,:], axis=0)
    midpoint = histogram.shape[0] // 2
    left_base = np.argmax(histogram[:midpoint])
    right_base = np.argmax(histogram[midpoint:]) + midpoint

    # Calculate the center of the lanes
    lane_center = (left_base + right_base) // 2
    img_center = lane_image.shape[1] // 2

    # Calculate the steering angle
    angle = np.arctan((lane_center - img_center) / lane_image.shape[0])
    angle = np.degrees(angle)
    return angle

# Main function
def main():
    video_path = '/Users/dxs6024/Downloads/test_video.mp4'
    img_height, img_width = 256, 256

    images = load_images_from_video(video_path, img_height, img_width)

    X_train, X_val = train_test_split(images, test_size=0.2, random_state=42)

    input_shape = (img_height, img_width, 3)
    autoencoder = build_autoencoder(input_shape)

    autoencoder.fit(X_train, X_train, epochs=50, batch_size=32, validation_data=(X_val, X_val))

    # Save the trained model
    autoencoder.save('lane_detection_autoencoder.h5')

    # Test on some images and visualize the results
    decoded_images = autoencoder.predict(X_val)

    for i in range(5):
        plt.subplot(3, 5, i + 1)
        plt.imshow(X_val[i])
        plt.title('Original')

        plt.subplot(3, 5, i + 6)
        plt.imshow(decoded_images[i])
        plt.title('Autoencoded')

        # Calculate and display the steering angle
        lane_image = (decoded_images[i] * 255).astype(np.uint8)
        angle = calculate_steering_angle(lane_image)
        print(f"Steering Angle: {angle:.2f} degrees")

        # Show the edges
        blurred = cv2.GaussianBlur(lane_image, (5, 5), 0)
        edges = cv2.Canny(blurred, 50, 150)
        plt.subplot(3, 5, i + 11)
        plt.imshow(edges, cmap='gray')
        plt.title('Edges')

    plt.show()

if __name__ == "__main__":
    main()

NameError: name 'video_path' is not defined

In [7]:
!pwd

/Users/dxs6024/Downloads
